# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample print(city_data_df.columns)
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,24.44,49,20,1.54,US,1689976033
1,1,ushuaia,-54.8000,-68.3000,-1.19,69,75,6.69,AR,1689975783
2,2,punta arenas,-53.1500,-70.9167,-2.16,92,100,2.47,CL,1689976632
3,3,greymouth,-42.4667,171.2000,5.22,75,14,3.63,NZ,1689976632
4,4,jatai,-17.8814,-51.7144,22.57,45,2,3.02,BR,1689976633


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.55,
    color="City"
)

# Display the map
city_data_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:

# Define the criteria for the ideal weather condition
# For example, let's assume you want to filter cities with a maximum temperature between 70 and 80 degrees Fahrenheit, and wind speed less than 10 mph.
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 21) &
    (city_data_df['Max Temp'] <= 31) &
    (city_data_df['Wind Speed'] <10 )
]

# Drop any rows with null values (if applicable)
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,24.44,49,20,1.54,US,1689976033
4,4,jatai,-17.8814,-51.7144,22.57,45,2,3.02,BR,1689976633
8,8,hilo,19.7297,-155.0900,27.84,74,100,3.09,US,1689976462
9,9,atafu village,-8.5421,-172.5159,28.79,70,64,4.83,TK,1689976041
10,10,nago,26.5881,127.9761,28.47,81,61,3.14,JP,1689976432


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,badger,US,64.8000,-147.5333,49,
1,ushuaia,AR,-54.8000,-68.3000,69,
2,punta arenas,CL,-53.1500,-70.9167,92,
3,greymouth,NZ,-42.4667,171.2000,75,
4,jatai,BR,-17.8814,-51.7144,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":10,
    "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
badger - nearest hotel: Hotel North Pole
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
punta arenas - nearest hotel: Hotel Lacolet
greymouth - nearest hotel: Recreation Hotel
jatai - nearest hotel: Hotel Rio Claro
port-aux-francais - nearest hotel: Keravel
adamstown - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
hilo - nearest hotel: Dolphin Bay Hotel
atafu village - nearest hotel: No hotel found
nago - nearest hotel: グリーンリッチホテル
puerto natales - nearest hotel: Dorotea Patagonia Hostel
thompson - nearest hotel: Thompson Inn
grytviken - nearest hotel: No hotel found
dolisie - nearest hotel: Residence Nelson
waitangi - nearest hotel: Hotel Chathams
tromso - nearest hotel: Clarion Collection Hotel Aurora
remire-montjoly - nearest hotel: Complexe Belova
vilyuchinsk - nearest hotel: Победа
nar'yan-mar - nearest hotel: Гостиница "Агат"
edinburgh of the seven seas - nearest hotel: No hotel found
menongue - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
0,badger,US,64.8000,-147.5333,49,Hotel North Pole
1,ushuaia,AR,-54.8000,-68.3000,69,Apart Hotel Aires del Beagle
2,punta arenas,CL,-53.1500,-70.9167,92,Hotel Lacolet
3,greymouth,NZ,-42.4667,171.2000,75,Recreation Hotel
4,jatai,BR,-17.8814,-51.7144,45,Hotel Rio Claro
...,...,...,...,...,...,...
557,brookings,US,44.3114,-96.7984,65,Quality Inn
558,lagoa,PT,39.0500,-27.9833,84,Casa da Madrinha
559,nguigmi,NE,14.2495,13.1092,53,Guest PAM
560,boende,CD,-0.2167,20.8667,64,Hotel Reference


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot with hover information
city_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.75,
    color="City",
    hover_cols=["City", "Country", "Hotel Name", "Humidity"],
    title="City Humidity with Nearby Hotel Information"

)

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)